Identify tweets related to UChicago and 3-4 universities of your choice

In [1]:
path = "/user/ivy2/Tweets/"
tweet_df = spark.read.json(path)

In [2]:
tweet_df.printSchema()

root
 |-- contributors: string (nullable = true)
 |-- coordinates: struct (nullable = true)
 |    |-- coordinates: array (nullable = true)
 |    |    |-- element: double (containsNull = true)
 |    |-- type: string (nullable = true)
 |-- created_at: string (nullable = true)
 |-- display_text_range: array (nullable = true)
 |    |-- element: long (containsNull = true)
 |-- entities: struct (nullable = true)
 |    |-- hashtags: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- indices: array (nullable = true)
 |    |    |    |    |-- element: long (containsNull = true)
 |    |    |    |-- text: string (nullable = true)
 |    |-- media: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- additional_media_info: struct (nullable = true)
 |    |    |    |    |-- description: string (nullable = true)
 |    |    |    |    |-- embeddable: boolean (nullable = true)
 |    |    |    |    |-- monetizable: bo

In [3]:
tweet_filt = tweet_df.where(tweet_df['text'].contains('harvard') | tweet_df['text'].contains('Harvard')\
                            | tweet_df['text'].contains('uchicago') | tweet_df['text'].contains('UChicago')\
                            | tweet_df['text'].contains('uChicago') | tweet_df['text'].contains('U of C')\
                            | tweet_df['text'].contains('university of chicago')\
                            | tweet_df['text'].contains('University of Chicago')\
                            | tweet_df['text'].contains('stanford') | tweet_df['text'].contains('Stanford')\
                            | tweet_df['text'].contains('ucla') | tweet_df['text'].contains('UCLA')\
                            | tweet_df['text'].contains('university of california los angeles') | tweet_df['text'].contains('u.c.l.a')\
                            | tweet_df['text'].contains('University of California Los Angeles'))


In [4]:
tweet_filt = tweet_filt.withColumn("screen_name", tweet_filt["user.screen_name"]).\
withColumn("followers_count", tweet_filt["user.followers_count"])

In [5]:
tweet_df_filt = tweet_filt.select\
(["created_at", "favorite_count", "favorited","geo","id","lang","reply_count", "retweet_count","retweeted","text","screen_name","followers_count"])


In [6]:
from pyspark.sql.functions import *
schools = when(col("text").contains('harvard') | col("text").contains('Harvard'), 'Harvard').\
when(col("text").contains('stanford') | col("text").contains('Stanford'), 'Stanford').\
when(col("text").contains('uchicago') | col("text").contains('UChicago') | col("text").contains('uChicago')\
     | col("text").contains('U of C') | col("text").contains('university of chicago')\
     | col("text").contains('University of Chicago'), 'University of Chicago').\
when(col("text").contains('ucla') | col("text").contains('UCLA') | col("text").contains('university of california los angeles')\
     | col("text").contains('University of California Los Angeles') | col("text").contains('u.c.l.a'), 'UCLA').otherwise(0)

tweet_df_filt = tweet_df_filt.withColumn("schools", schools)

In [7]:
tweet_df_filt.groupby('schools').count().show()

+--------------------+------+
|             schools| count|
+--------------------+------+
|            Stanford|340575|
|                UCLA| 29045|
|University of Chi...|133563|
|             Harvard|976501|
+--------------------+------+



In [8]:
tweet_df_filt = tweet_df_filt.withColumn("day_of_week", split(col("created_at"), "\\ ").getItem(0))\
                .withColumn("month", split(col("created_at"), "\\ ").getItem(1))\
                .withColumn("day", split(col("created_at"), "\\ ").getItem(2))\
                .withColumn("time", split(col("created_at"), "\\ ").getItem(3))\
                .withColumn("code", split(col("created_at"), "\\ ").getItem(4))\
                .withColumn("year", split(col("created_at"), "\\ ").getItem(5))\
                .withColumn("rt_vs_original", when(col("text").contains('RT @')\
                                                           | col("text").contains('rt @'), 'rt').otherwise('original'))

Discard irrelevant tweets

In [11]:
import os
#os.chdir('C:\\Users\\Glynis\\Documents\\MSc A U Chicago\\big data')
savetest = tweet_df_filt.groupby('month','day','year','rt_vs_original','schools','screen_name').\
            agg(max('followers_count'),sum('favorite_count'),sum('reply_count'),sum('retweet_count'), count('*'))
#    (["created_at", "favorite_count", "favorited","geo","id","lang","reply_count", "retweet_count","retweeted","text","screen_name","followers_count"])
savetest.show()

+-----+---+----+--------------+--------+---------------+--------------------+-------------------+----------------+------------------+--------+
|month|day|year|rt_vs_original| schools|    screen_name|max(followers_count)|sum(favorite_count)|sum(reply_count)|sum(retweet_count)|count(1)|
+-----+---+----+--------------+--------+---------------+--------------------+-------------------+----------------+------------------+--------+
|  Sep| 04|2017|      original| Harvard|       suzieg81|                 392|                  0|               0|                 0|       1|
|  Sep| 04|2017|            rt|Stanford|        eex8692|                 229|                  0|               0|                 0|       1|
|  Dec| 12|2017|            rt| Harvard|starbucksgirl51|                 473|                  0|               0|                 0|       1|
|  Dec| 12|2017|            rt| Harvard| susanjfeingold|                 592|                  0|               0|                 0|       1|

In [17]:
savetest.groupby('screen_name').agg(sum('sum(retweet_count)')).show()

+---------------+-----------------------+
|    screen_name|sum(sum(retweet_count))|
+---------------+-----------------------+
|         mara1y|                      0|
|CHERIANMCCARTNE|                      0|
|    marienick10|                      0|
|continentaldrop|                      0|
|     moreprod43|                      0|
|    EricSchultz|                      0|
|  EmmMacfarlane|                      0|
|   bobgreenberg|                      0|
|    onthecoasts|                      0|
|    Nuaim230271|                      0|
|  subsublibrary|                      0|
| teamsteepNdeep|                      0|
|FieldsieScience|                      0|
|         edwebb|                      0|
|  DeepuJanasena|                      0|
|   snoopylover2|                      0|
|OrlandoMelgar48|                      0|
|   badger_house|                      0|
|    claudiaordn|                      0|
|   defauconberg|                      0|
+---------------+-----------------

In [18]:
pdf = savetest.toPandas()

In [19]:
pdf.head(3)

,month,day,year,rt_vs_original,schools,screen_name,max(followers_count),sum(favorite_count),sum(reply_count),sum(retweet_count),count(1)
0,Sep,27,2017,rt,University of Chicago,gcrolee,63,0,0.0,0,2
1,Mar,14,2018,rt,University of Chicago,ChiConfidential,5471,0,0.0,0,1
2,Mar,14,2018,rt,Stanford,seerah_A,333,0,0.0,0,1


In [21]:
import os
os.chdir('/home/gsmyers/')


In [24]:
pdf.to_csv('finalprojtest.csv')

In [26]:
geosave = tweet_df_filt.groupby('geo','rt_vs_original','schools').\
            agg(avg('followers_count'),sum('favorite_count'),sum('reply_count'),sum('retweet_count'), count('*'))

In [27]:
geosave.show(5)

+--------------------+--------------+--------+--------------------+-------------------+----------------+------------------+--------+
|                 geo|rt_vs_original| schools|avg(followers_count)|sum(favorite_count)|sum(reply_count)|sum(retweet_count)|count(1)|
+--------------------+--------------+--------+--------------------+-------------------+----------------+------------------+--------+
|[WrappedArray(37....|      original|Stanford|              1506.0|                  0|            null|                 0|       1|
|[WrappedArray(40....|      original| Harvard|              2927.0|                  0|               0|                 0|       1|
|[WrappedArray(33....|      original|    UCLA|              4089.0|                  0|               0|                 0|       1|
|[WrappedArray(3.1...|      original| Harvard|                38.0|                  0|               0|                 0|       1|
|[WrappedArray(37....|      original|Stanford|               108.0|  

In [28]:
geosave.toPandas().to_csv('finalprojgeo.csv')

In [29]:
timeofday = tweet_df_filt.groupby('time','rt_vs_original','schools').\
            agg(avg('followers_count'),sum('favorite_count'),sum('reply_count'),sum('retweet_count'), count('*'))

In [30]:
timeofday.toPandas().to_csv('finalprojtime.csv')

In [33]:
! ls -l '/home/gsmyers/'

total 332928
drwxrwxr-x 3 gsmyers gsmyers     4096 May 18 17:00 bigdata
-rw-rw-rw- 1 gsmyers gsmyers 30682276 Oct 11  2004 BX-Book-Ratings.csv
-rw-rw-rw- 1 gsmyers gsmyers 77787439 Oct 11  2004 BX-Books.csv
-rwxr-xr-x 1 gsmyers gsmyers 26085508 Apr 30 13:45 BX-CSV-Dump.zip
-rw-rw-rw- 1 gsmyers gsmyers 12284157 Oct 11  2004 BX-Users.csv
-rw-r--r-- 1 gsmyers gsmyers    21168 May 24 11:55 Capstone+-+Multinomial+Logistic+Classifier (1).ipynb
-rw-rw-r-- 1 gsmyers gsmyers       22 Nov  7  2017 DDAIFall2017-TestFTP.txt
drwxr-xr-x 2 gsmyers gsmyers     4096 Jan  6 12:48 Desktop
-rw-r--r-- 1 gsmyers gsmyers   155329 May 24 11:36 Final Project.ipynb
-rw-r--r-- 1 gsmyers gsmyers   113170 May 29 11:03 Final Project v2-Copy1.ipynb
-rw-r--r-- 1 gsmyers gsmyers   113170 May 26 12:29 Final Project v2.ipynb
-rw-r--r-- 1 gsmyers gsmyers   117587 May 30 18:59 Final Project v3.ipynb
-rw-r--r-- 1 gsmyers gsmyers    90080 May 30 18:59 finalprojgeo.csv
-rw-r--r-- 1 gsmyers gsmyers   160120 May 30 18:26 final